# Process results

This notebook analyzes the matching decisions by the simulation.py and summarizes a few metrics

In [1]:
import pandas as pd
import numpy as np

In [2]:
recipients = pd.read_csv('data/anon_recipients.csv',index_col=0,parse_dates=['date'])
recipients = recipients.set_index(['rec_id','date']).stack().to_frame().reset_index()
recipients.columns=['rec_id','date','ppe','qty']
recipients = recipients[recipients['qty']>0]

In [3]:
recipients

,rec_id,date,ppe,qty
3,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0
9,rec0,2020-04-02 16:27:00+00:00,respirators,10000.0
10,rec0,2020-04-02 16:27:00+00:00,gowns,1000.0
12,rec0,2020-04-02 16:27:00+00:00,faceShields,5000.0
19,rec1,2020-04-02 16:35:00+00:00,disinfectingWipes,4.0
...,...,...,...,...
114381,rec6354,2020-07-17 08:14:58.249000+00:00,respirators,50.0
114389,rec6354,2020-07-17 08:14:58.249000+00:00,rec_req_id,6354.0
114395,rec6355,2020-07-17 14:25:54.605000+00:00,disposableBooties,1.0
114399,rec6355,2020-07-17 14:25:54.605000+00:00,respirators,1.0


In [4]:
donors = pd.read_csv('data/anon_donors.csv',index_col=0,parse_dates=['date'])
donors

,don_id,date,ppe,qty,don_req_id
469,don0,2020-04-09 13:08:00+00:00,faceShields,10.0,0
470,don1,2020-04-09 13:36:00+00:00,faceShields,1.0,1
327,don2,2020-04-09 13:53:00+00:00,faceShields,3000.0,2
328,don3,2020-04-09 14:24:00+00:00,faceShields,5.0,3
329,don71,2020-04-09 17:52:00+00:00,faceShields,10.0,4
...,...,...,...,...,...
20,don994,2020-07-17 21:52:59.717000+00:00,respirators,500.0,992
1658,don994,2020-07-17 21:52:59.717000+00:00,faceShields,500.0,993
13,don994,2020-07-17 21:52:59.717000+00:00,coveralls,1000.0,994
1708,don995,2020-07-18 12:11:44.881000+00:00,handmadeMasks,50.0,995


In [7]:
import pickle
distance = pickle.load( open( "data/anon_distance_matrix.p", "rb" ) )
distance

AttributeError: 'DataFrame' object has no attribute '_data'

In [8]:
decisions = pd.read_csv('decisions.csv',index_col=0,parse_dates=['date'])
decisions

,date,don_id,rec_id,ppe,qty,distance
0,2020-04-09 16:26:00+00:00,don0,rec502,faceShields,10.0,10.359351
1,2020-04-09 16:26:00+00:00,don1,rec1173,faceShields,1.0,10.102131
2,2020-04-09 16:26:00+00:00,don3,rec1283,faceShields,5.0,2.175027
3,2020-04-16 16:26:00+00:00,don10,rec1261,faceShields,20.0,3.113544
4,2020-04-16 16:26:00+00:00,don11,rec1571,faceShields,100.0,8.081337
...,...,...,...,...,...,...
802,2020-07-16 16:26:00+00:00,don966,rec1742,faceShields,30.0,0.000000
803,2020-07-16 16:26:00+00:00,don970,rec5118,faceShields,15.0,3.082334
804,2020-07-16 16:26:00+00:00,don962,rec4300,handSanitizer,3.0,0.000000
805,2020-07-16 16:26:00+00:00,don969,rec853,disinfectingWipes,3.0,1.978864


In [9]:
decisions.groupby('don_id').size().sort_values()

don_id
don0      1
don637    1
don638    1
don639    1
don64     1
         ..
don609    3
don625    3
don494    4
don824    4
don835    5
Length: 719, dtype: int64

In [10]:
ppes_to_consider = set(donors.ppe.unique())
ppes_to_consider = ppes_to_consider.intersection(set(recipients.ppe.unique()))

# For each ppe, compute stats

Fill rate

In [11]:
merg = recipients.merge(decisions,on=['rec_id','ppe'],how='left',suffixes=('_rec','_decision'))
merg

,rec_id,date_rec,ppe,qty_rec,date_decision,don_id,qty_decision,distance
0,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0,NaT,NaN,NaN,NaN
1,rec0,2020-04-02 16:27:00+00:00,respirators,10000.0,NaT,NaN,NaN,NaN
2,rec0,2020-04-02 16:27:00+00:00,gowns,1000.0,NaT,NaN,NaN,NaN
3,rec0,2020-04-02 16:27:00+00:00,faceShields,5000.0,NaT,NaN,NaN,NaN
4,rec1,2020-04-02 16:35:00+00:00,disinfectingWipes,4.0,2020-04-23 16:26:00+00:00,don114,4.0,1.910903
...,...,...,...,...,...,...,...,...
37399,rec6354,2020-07-17 08:14:58.249000+00:00,respirators,50.0,NaT,NaN,NaN,NaN
37400,rec6354,2020-07-17 08:14:58.249000+00:00,rec_req_id,6354.0,NaT,NaN,NaN,NaN
37401,rec6355,2020-07-17 14:25:54.605000+00:00,disposableBooties,1.0,NaT,NaN,NaN,NaN
37402,rec6355,2020-07-17 14:25:54.605000+00:00,respirators,1.0,NaT,NaN,NaN,NaN


In [12]:
merg['delay'] = (merg['date_decision'] - merg['date_rec']).dt.days
#merg = merg[merg['delay'] < threshold_days]

In [13]:
merg

,rec_id,date_rec,ppe,qty_rec,date_decision,don_id,qty_decision,distance,delay
0,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0,NaT,NaN,NaN,NaN,NaN
1,rec0,2020-04-02 16:27:00+00:00,respirators,10000.0,NaT,NaN,NaN,NaN,NaN
2,rec0,2020-04-02 16:27:00+00:00,gowns,1000.0,NaT,NaN,NaN,NaN,NaN
3,rec0,2020-04-02 16:27:00+00:00,faceShields,5000.0,NaT,NaN,NaN,NaN,NaN
4,rec1,2020-04-02 16:35:00+00:00,disinfectingWipes,4.0,2020-04-23 16:26:00+00:00,don114,4.0,1.910903,20.0
...,...,...,...,...,...,...,...,...,...
37399,rec6354,2020-07-17 08:14:58.249000+00:00,respirators,50.0,NaT,NaN,NaN,NaN,NaN
37400,rec6354,2020-07-17 08:14:58.249000+00:00,rec_req_id,6354.0,NaT,NaN,NaN,NaN,NaN
37401,rec6355,2020-07-17 14:25:54.605000+00:00,disposableBooties,1.0,NaT,NaN,NaN,NaN,NaN
37402,rec6355,2020-07-17 14:25:54.605000+00:00,respirators,1.0,NaT,NaN,NaN,NaN,NaN


In [14]:
res = merg.groupby(['rec_id','ppe']).agg({'qty_rec':min,'qty_decision':sum}).reset_index()
res['total_fill_rate'] = res.qty_decision / res.qty_rec
res

,rec_id,ppe,qty_rec,qty_decision,total_fill_rate
0,rec0,faceShields,5000.0,0.0,0.0
1,rec0,gowns,1000.0,0.0,0.0
2,rec0,nitrileGloves,10000.0,0.0,0.0
3,rec0,respirators,10000.0,0.0,0.0
4,rec1,disinfectingWipes,4.0,4.0,1.0
...,...,...,...,...,...
37299,rec998,respirators,50.0,0.0,0.0
37300,rec998,thermometers,2.0,0.0,0.0
37301,rec999,faceShields,20.0,0.0,0.0
37302,rec999,rec_req_id,999.0,0.0,0.0


In [15]:
res

,rec_id,ppe,qty_rec,qty_decision,total_fill_rate
0,rec0,faceShields,5000.0,0.0,0.0
1,rec0,gowns,1000.0,0.0,0.0
2,rec0,nitrileGloves,10000.0,0.0,0.0
3,rec0,respirators,10000.0,0.0,0.0
4,rec1,disinfectingWipes,4.0,4.0,1.0
...,...,...,...,...,...
37299,rec998,respirators,50.0,0.0,0.0
37300,rec998,thermometers,2.0,0.0,0.0
37301,rec999,faceShields,20.0,0.0,0.0
37302,rec999,rec_req_id,999.0,0.0,0.0


In [16]:
res.groupby('ppe')['total_fill_rate'].mean()

ppe
babyMonitors         0.005286
bodyBags             0.005435
coveralls            0.023564
disinfectingWipes    0.002841
disposableBooties    0.006105
faceShields          0.046250
gowns                0.000925
handSanitizer        0.003906
handmadeMasks        0.050814
nitrileGloves        0.006539
paprShield           0.000000
rec_req_id           0.000000
respirators          0.012295
safetyGlasses        0.008809
safetyGoggles        0.011034
surgicalCaps         0.001350
surgicalMasks        0.020832
thermometers         0.000722
Name: total_fill_rate, dtype: float64